In [1]:
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer

In [2]:
dataset = pd.read_csv('data/keywords_.csv', sep=';')

In [3]:
df = pd.DataFrame(dataset)
df

,'AdGroupId','CampaignId','Id','Keyword','ServingStatus','State','Status','StrategyPriority'
0,3767436593,42244009,16348493015,'фас тарифы -жкх','ELIGIBLE','ON','ACCEPTED','NORMAL'
1,3767436593,42244009,16348696779,'фас тарифы жкх','ELIGIBLE','ON','ACCEPTED','NORMAL'
2,3767470344,42244009,16348696780,'определение суд экспертизы','ELIGIBLE','ON','ACCEPTED','NORMAL'
3,3767470344,42244009,16348696782,'определение суда +о назначении экспертизы','ELIGIBLE','ON','ACCEPTED','NORMAL'
4,3767470345,42244009,16348696783,'жкх последние изменения','ELIGIBLE','ON','ACCEPTED','NORMAL'
...,...,...,...,...,...,...,...,...
3995,3126857265,26790451,13056964783,'DC Shoes Banshee','ELIGIBLE','ON','ACCEPTED','NORMAL'
3996,3126857265,26790451,13056964790,'Shoes Banshee -dc','ELIGIBLE','ON','ACCEPTED','NORMAL'
3997,3126857265,26790451,13056964798,'686 Mannual Standard -w','ELIGIBLE','ON','ACCEPTED','NORMAL'
3998,3126857265,26790451,13056964806,'686 Mannual Standard W','ELIGIBLE','ON','ACCEPTED','NORMAL'


In [4]:
df['\'Keyword\''] = list(map(lambda x: re.sub(r'\s-[a-zA-Zа-яА-Я!+0-9]+', '', x), df['\'Keyword\'']))

In [5]:
df

,'AdGroupId','CampaignId','Id','Keyword','ServingStatus','State','Status','StrategyPriority'
0,3767436593,42244009,16348493015,'фас тарифы','ELIGIBLE','ON','ACCEPTED','NORMAL'
1,3767436593,42244009,16348696779,'фас тарифы жкх','ELIGIBLE','ON','ACCEPTED','NORMAL'
2,3767470344,42244009,16348696780,'определение суд экспертизы','ELIGIBLE','ON','ACCEPTED','NORMAL'
3,3767470344,42244009,16348696782,'определение суда +о назначении экспертизы','ELIGIBLE','ON','ACCEPTED','NORMAL'
4,3767470345,42244009,16348696783,'жкх последние изменения','ELIGIBLE','ON','ACCEPTED','NORMAL'
...,...,...,...,...,...,...,...,...
3995,3126857265,26790451,13056964783,'DC Shoes Banshee','ELIGIBLE','ON','ACCEPTED','NORMAL'
3996,3126857265,26790451,13056964790,'Shoes Banshee','ELIGIBLE','ON','ACCEPTED','NORMAL'
3997,3126857265,26790451,13056964798,'686 Mannual Standard','ELIGIBLE','ON','ACCEPTED','NORMAL'
3998,3126857265,26790451,13056964806,'686 Mannual Standard W','ELIGIBLE','ON','ACCEPTED','NORMAL'


In [14]:
df_result = pd.DataFrame({'Keyword_x' : [], 'AdGroupId_x' : [], 'Keyword_y' : [], 'AdGroupId_y' : [], 'crossed' : []})

In [ ]:
tokenizer = RegexpTokenizer(r'[a-zA-Zа-яА-Я+!0-9]+')
tokens = list(map(lambda x: tokenizer.tokenize(x.lower()), df['\'Keyword\'']))
for i in df.index:
    agi = df['\'AdGroupId\''][i]
    series = df.where(df['\'AdGroupId\''] != agi)
    series.dropna(inplace=True)
    inds = [s for s in series.index if len([word for word in tokens[i] if word in tokens[s]]) > 1]
    if len(inds) > 0:
        kws_x = [df['\'Keyword\''][i]] * len(inds)
        agi_x = [df['\'AdGroupId\''][i]] * len(inds)
        df_search = df.loc[df.index.isin(inds)]
        kws_y = df_search['\'Keyword\''].tolist()
        agi_y = df_search['\'AdGroupId\''].tolist()
        subsets = list(list())
        for s in inds:
            subsets.append([w for w in tokens[i] if w in tokens[s]])
        subsets = [', '.join(map(str, w)) for w in subsets]
        df_add = pd.DataFrame({'Keyword_x' : [], 'AdGroupId_x' : [], 'Keyword_y' : [], 'AdGroupId_y' : [], 'crossed' : []})
        df_add['Keyword_x'] = kws_x
        df_add['AdGroupId_x'] = agi_x 
        df_add['Keyword_y'] = kws_y
        df_add['AdGroupId_y'] = agi_y
        df_add['crossed'] =  subsets
        df_result = pd.concat([df_result, df_add])
        


In [16]:
df_result[['AdGroupId_x', 'AdGroupId_y']] = df_result[['AdGroupId_x', 'AdGroupId_y']].astype('int64')

In [18]:
df_result.to_csv(r'data/keywords_ext.csv', index=None)

In [17]:
df_result

,Keyword_x,AdGroupId_x,Keyword_y,AdGroupId_y,crossed
0,'определение суд экспертизы',3767470344,'суд экспертизы рф',3767470352,"суд, экспертизы"
0,'изменения +в жкх',3767470345,'жкх подали +в суд',3767470351,"+в, жкх"
0,'оплата жкх',3767470346,'оплата коммунальных услуг жкх',3767470348,"оплата, жкх"
0,'стоимость услуг жкх',3767470346,'оплата коммунальных услуг жкх',3767470348,"услуг, жкх"
1,'стоимость услуг жкх',3767470346,'аудиторская компания стоимость услуг',3767585626,"стоимость, услуг"
...,...,...,...,...,...
7,'Кофр +на горник',3100693367,'кофр +на снегоход буран',3100843862,"кофр, +на"
8,'Кофр +на горник',3100693367,'кофр +на буран',3100843862,"кофр, +на"
9,'Кофр +на горник',3100693367,'Кофр +на кота',3100693378,"кофр, +на"
10,'Кофр +на горник',3100693367,'Кофр на горник',3141292985,"кофр, горник"
